In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import glob
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
import chardet

def encode_detect(file):
    with open(file, 'rb') as f:
        return chardet.detect(f.read())['encoding']

def read_file(file):
    print("Reading file: {}".format(file))
    return pd.read_csv(file, encoding=encode_detect(file))


In [6]:
df_na_check = read_file('data/all/附件6.csv')  # 读取核酸采样检测信息表

file_list = glob.glob('data/all/附件5/*.csv')
df_scan = pd.concat([read_file(file) for file in file_list])  # 读取场所扫码信息表
df_scan['create_time'] =  pd.to_datetime(df_scan['create_time'])

df_people = read_file('data/all/附件2.csv')  # 读取人员信息表
df_place = read_file('data/all/附件3.csv')  # 读取场所信息表

df_self_check = read_file('data/all/附件4.csv')  # 读取个人自查上报信息表
df_self_check['dump_time'] = pd.to_datetime(df_self_check['dump_time'])
# 提交示例1
result1 = read_file('data/exp/附件1/result1.csv')
# 提交示例2
result2 = read_file('data/exp/附件1/result2.csv')

Reading file: data/all/附件6.csv
Reading file: data/all/附件5\20221001.csv
Reading file: data/all/附件5\20221002.csv
Reading file: data/all/附件5\20221003.csv
Reading file: data/all/附件5\20221004.csv
Reading file: data/all/附件5\20221005.csv
Reading file: data/all/附件5\20221006.csv
Reading file: data/all/附件5\20221007.csv
Reading file: data/all/附件5\20221008.csv
Reading file: data/all/附件5\20221009.csv
Reading file: data/all/附件5\20221010.csv
Reading file: data/all/附件5\20221011.csv
Reading file: data/all/附件5\20221012.csv
Reading file: data/all/附件5\20221013.csv
Reading file: data/all/附件5\20221014.csv
Reading file: data/all/附件5\20221015.csv
Reading file: data/all/附件5\20221016.csv
Reading file: data/all/附件5\20221017.csv
Reading file: data/all/附件5\20221018.csv
Reading file: data/all/附件5\20221019.csv
Reading file: data/all/附件5\20221020.csv
Reading file: data/all/附件5\20221021.csv
Reading file: data/all/附件5\20221022.csv
Reading file: data/all/附件5\20221023.csv
Reading file: data/all/附件5\20221024.csv
Reading f

In [6]:
# 查看提交示例
result1.head()

,序号,密接者ID,密接日期,密接场所ID,阳性人员ID
0,1,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN


In [6]:
# 查看提交示例
result2.head()

,序号,次密接者ID,次密接日期,次密接场所ID,密接者ID
0,1,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN


In [8]:
# 数据描述性统计
def summary_stats_table(data):
    '''
    a function to summerize all types of data
    分类型按列的数据分布与异常值统计
    '''
    # count of nulls
    # 空值数量
    missing_counts = pd.DataFrame(data.isnull().sum())
    missing_counts.columns = ['count_null']

    # numeric column stats
    # 数值列数据分布统计
    num_stats = data.select_dtypes(include=['int64','float64']).describe().loc[['count','min','max','25%','50%','75%']].transpose()
    num_stats['dtype'] = data.select_dtypes(include=['int64','float64']).dtypes.tolist()

    # non-numeric value stats
    # 非数值列数据分布统计
    non_num_stats = data.select_dtypes(exclude=['int64','float64']).describe().transpose()
    non_num_stats['dtype'] = data.select_dtypes(exclude=['int64','float64']).dtypes.tolist()
    non_num_stats = non_num_stats.rename(columns={"first": "min", "last": "max"})

    # merge all
    # 聚合结果
    stats_merge = pd.concat([num_stats, non_num_stats], axis=0, join='outer', ignore_index=False, keys=None,
              levels=None, names=None, verify_integrity=False, copy=True, sort=False).fillna("").sort_values('dtype')

    column_order = ['dtype', 'count', 'count_null','unique','min','max','25%','50%','75%','top','freq']
    summary_stats = pd.merge(stats_merge, missing_counts, left_index=True, right_index=True, sort=False)[column_order]
    return(summary_stats)

In [9]:
summary_stats_table(df_people)

,dtype,count,count_null,unique,min,max,25%,50%,75%,top,freq
user_id,int64,99996.0,0,,1.0,99996.0,24999.75,49998.5,74997.25,,
age,float64,99985.0,11,,-7.0,982.0,22.0,37.0,51.0,,
openid,object,98028.0,1968,98028,,,,,,oESgL4Kw2tG7yZWlxdgHv4TkSgFN8Ibza6,1
gender,object,99965.0,31,2,,,,,,男,52656
nation,object,99621.0,375,10,,,,,,汉族,95942
birthdate,object,99978.0,18,26959,,,,,,2005/11/6,16
create_time,object,99996.0,0,99918,,,,,,2022-02-12 08:50:45,2


**数据说明**
- user_id：人员 ID，用于唯一标识一个人员。
- openid：微信 OpenID，用于关联该人员的微信账号信息。
- gender：人员的性别，可选值为“男”或“女”。
- nation：人员所属的民族，如汉族、蒙古族、藏族等。
- age：人员的年龄，以整数表示。
- birthdate：人员的出生日期，格式一般为“YYYY-MM-DD”。
- create_time：该记录的创建时间，用于记录人员信息的更新时间。


**以下分析结果均基于全量数据**

用户总共有99996条数据。

- 年龄区间是[-7，982]

    Tips:年龄跨度异常大，可能存在异常值，例如负数的年龄。需要进一步检查并处理异常值。
    Tips:年龄跨度比较大，自然而然，我们可以根据年龄做特征工程。

- 在gender中，总共有2个类别，但是实际数据中有31个缺失值。

    Tips:需要处理缺失值，例如用众数填充或将其标记为“未知”。

- nation民族总共有10个类别，其中汉族占比最高，达到95942。

    Tips:在全量数据中，可以考虑将民族特征进行独热编码或分组处理，以便于分析和建模。
    Tips:如果后续需要用到该列进行聚合分析或特征工程，可以在Baseline中写好动态的代码。


- birthdate列中有18个缺失值，需要处理这些缺失值。

    Tips:可以考虑使用平均值、中位数或众数等方法填充缺失值。

- create_time列没有缺失值，但是需要注意数据类型为object。

    Tips:将create_time转换为日期类型，以便于进一步处理和分析。
    Tips:注意关注时间的始末，与其它相关联的时间进行比较，这样可以挖掘出更多信息或筛选出一些异常情况。

根据上述分析，我们可以得出以下建议：

检查并处理年龄列中的异常值，例如负数。
处理gender、birthdate等列中的缺失值，例如使用众数填充或将其标记为“未知”。
将日期相关列（如birthdate和create_time）的数据类型转换为日期类型。
对民族特征进行独热编码或分组处理，以便于分析和建模。
考虑根据已有数据生成新的特征，例如年龄段或根据birthdate和create_time计算用户的年龄。


- birthdate和create_time在这里都是对应着50个不一样的时间

    Tips:注意关注时间的始末，与其它相关联的时间进行比较，这样可以挖掘出更多信息或筛选出一些异常情况。

         在全量数据中，时间大概率是有重复值的，也要考虑重复时间是否对解题有一定的影响亦或者重复时间的含义。


In [10]:
summary_stats_table(df_place)

,dtype,count,count_null,unique,min,max,25%,50%,75%,top,freq
grid_point_id,int64,2947.0,0,,1.0,2947.0,737.5,1474.0,2210.5,,
x_coordinate,float64,2947.0,0,,34.79,29320.01,9973.45,12565.56,15029.45,,
y_coordinate,float64,2947.0,0,,10.76,29710.22,9521.09,12054.57,14700.45,,
name,object,2947.0,0,2947,,,,,,宾馆旅店1,1
point_type,object,2947.0,0,18,,,,,,娱乐,400
create_time,object,2947.0,0,2942,,,,,,2020/5/23 6:08,2


**数据说明**
- grid_point_id：场所 ID，用于唯一标识一个场所。
- name：场所的名称，如公司、餐厅、超市等。
- point_type：场所的类型，如商业、娱乐、文化、医疗等。
- x_coordinate：场所的 X 坐标，以米为单位，用于表示场所在地图上的位置。
- y_coordinate：场所的 Y 坐标，以米为单位，用于表示场所在地图上的位置。
- create_time：该记录的创建时间，用于记录场所信息的更新时间。


**以下分析结果均基于全量数据**

场所总共有2947条数据。
- 坐标区间：
    x_coordinate范围在[34.79, 29320.01]之间。
    y_coordinate范围在[10.76, 29710.22]之间。
    Tips:可以根据X、Y坐标对其它特征进行可视化

- 在name列中，共有2947个不同的名称。

    Tips:可以考虑对场所名称进行文本分析，提取关键词，以了解场所的具体业务或属性。

- 在point_type中，总共有18个类别，娱乐类场所数量最多，共有400个。

    Tips:可以对不同类型的场所进行分析，以了解各类场所的分布情况和特点。可以根据这一列特征做更多的数据分析，或许还可以进行特征工程。

- create_time列没有缺失值，但是需要注意数据类型为object。

    Tips:将create_time转换为日期类型，以便于进一步处理和分析。

根据上述分析，我们可以得出以下建议：

计算场所之间的距离，以便于分析不同类型场所的分布情况。
对场所名称进行文本分析，提取关键词，以了解场所的具体业务或属性。
对不同类型的场所进行分析，以了解各类场所的分布情况和特点。
将日期相关列（如create_time）的数据类型转换为日期类型。
考虑根据已有数据生成新的特征，例如基于场所类型的分组统计或根据坐标信息计算密度等。

In [11]:
summary_stats_table(df_self_check)

,dtype,count,count_null,unique,min,max,25%,50%,75%,top,freq
sno,int64,74736.0,0,,1.0,74736.0,18684.75,37368.5,56052.25,NaT,
user_id,int64,74736.0,0,,14.0,99992.0,24648.0,49721.0,74849.0,NaT,
symptom,int64,74736.0,0,,1.0,8.0,8.0,8.0,8.0,NaT,
nucleic_acid_result,int64,74736.0,0,,0.0,1.0,0.0,0.0,0.0,NaT,
resident_flag,int64,74736.0,0,,1.0,2.0,1.0,1.0,1.0,NaT,
x_coordinate,float64,74736.0,0,,0.6,19999.48,5010.8875,10001.76,15018.6775,NaT,
y_coordinate,float64,74736.0,0,,0.21,19999.91,4991.1975,9950.215,14929.6225,NaT,
dump_time,datetime64[ns],74736.0,0,73830,2022-10-01 07:01:18,2022-11-30 22:59:58,,,,2022-10-28 18:34:35,3


**数据说明**
- sno：序列号，用于唯一标识一条自查记录。
- user_id：人员 ID，对应于“人员信息表”中的 user_id，用于关联自查记录与相应的人员。
- x_coordinate：上报地点的 X 坐标，以米为单位，用于表示上报地点在地图上的位置。
- y_coordinate：上报地点的 Y 坐标，以米为单位，用于表示上报地点在地图上的位置。
- symptom：症状，用于记录自查者的症状情况。可选值为：1 发热、2 乏力、3 干咳、4 鼻塞、5 流涕、6 腹泻、7 呼吸困难、8 无症状。
- nucleic_acid_result：核酸检测结果，用于记录自查者的核酸检测情况。可选值为：0 阴性、1 阳性、2 未知（非必填）。
- resident_flag：是否常住居民，用于记录自查者的居住情况。可选值为：0 未知、1 是、2 否。
- dump_time：上报时间，用于记录自查记录的上报时间。


**以下分析结果均基于全量数据**

自查记录总共有74736条数据。

- 这里的X，Y坐标和上表的并不一样，可以挖掘一下两者的区别
    坐标区间：

    x_coordinate范围在[0.6, 19999.48]之间。
    y_coordinate范围在[0.21, 19999.91]之间。
    Tips:可以根据坐标信息计算上报地点之间的距离，以便于分析不同地区的自查情况。

- 在symptom列中，共有8种症状。

    Tips:可以对不同症状进行分析，以了解自查者的症状分布情况。可以考虑先写好数据分析可视化的代码。

    这一列特征还有一个特点是，在特征工程的时候，可以很好的和其它特征衍生出很多可解释性的交叉特征（eg:symptom-nucleic_acid_result）

    在nucleic_acid_result中，总共有3种结果，其中阴性结果占多数。
    在resident_flag中，共有3种情况，其中常住居民占多数。
- nucleic_acid_result,resident_flag同理


- dump_time列没有缺失值，数据类型为datetime64[ns]。

    Tips:可以分析不同时间段的自查情况，例如按天、周或月统计。

根据上述分析，我们可以得出以下建议：

计算上报地点之间的距离，以便于分析不同地区的自查情况。
对不同症状进行分析，以了解自查者的症状分布情况。
对核酸检测结果进行分析，了解自查者中阳性和阴性的比例。
分析常住居民和非常住居民的自查情况，以了解不同人群的自查行为。
分析不同时间段的自查情况，例如按天、周或月统计。

In [12]:
summary_stats_table(df_scan)

,dtype,count,count_null,unique,min,max,25%,50%,75%,top,freq
sno,int64,24353780.0,0,,1.0,415327.0,99811.0,199622.0,299432.0,NaT,
grid_point_id,int64,24353780.0,0,,1.0,2947.0,751.0,1503.0,2225.0,NaT,
user_id,int64,24353780.0,0,,1.0,99996.0,24996.0,50053.0,75017.0,NaT,
temperature,float64,24353780.0,0,,35.7,37.2,36.1,36.4,36.8,NaT,
create_time,datetime64[ns],24353780.0,0,4538584,2022-10-01 05:30:00,2022-12-01 04:47:59,,,,2022-10-21 11:55:43,26


**数据说明**
- sno：序列号，用于唯一标识一条扫码记录。
- grid_point_id：场所 ID，对应于“场所信息表”中的 grid_point_id，用于关联扫码记录与相应的场所。
- user_id：人员 ID，对应于“人员信息表”中的 user_id，用于关联扫码记录与相应的人员。
- temperature：体温，用于记录扫码者的体温情况。
- create_time：扫码记录时间，用于记录扫码记录的时间戳。


**以下分析结果均基于全量数据**

扫码记录总共有24,353,780条数据。

- 体温范围在[35.7, 37.2]之间。

    Tips:可以对不同体温范围的人数进行统计，以了解体温分布情况。在实际数据中，可能会有更高或更低的体温值，需要在分析时考虑这些异常值。

- create_time（扫码记录时间）:

    Tips: 可以分析不同时间段的扫码情况，例如按天、周或月统计。此外，可以将扫码记录时间与个人自查上报信息表的上报时间以及采样日期进行比较，以了解扫码行为与自查行为之间的关联。

根据上述分析，我们可以得出以下建议：

对不同体温范围的人数进行统计，以了解体温分布情况。注意考虑实际数据中可能存在的异常值。
分析不同时间段的扫码情况，例如按天、周或月统计。
将扫码记录时间与个人自查上报信息表的上报时间以及采样日期进行比较，以了解扫码行为与自查行为之间的关联。
结合场所信息和人员信息，分析不同场所、人群的扫码行为，以了解场所类型和人群特征对扫码行为的影响。

In [13]:
summary_stats_table(df_na_check)

,dtype,count,count_null,unique,min,max,25%,50%,75%,top,freq
sno,int64,3967851.0,0,,1.0,3985187.0,991528.5,1986191.0,2984163.5,,
user_id,int64,3967851.0,0,,1.0,99996.0,24994.0,50001.0,74992.0,,
grid_point_id,int64,3967851.0,0,,58.0,2891.0,766.0,1475.0,2183.0,,
cysj,object,3967851.0,0,1121300,,,,,,2022-11-20 11:39:55,16
jcsj,object,3967851.0,0,787289,,,,,,2022-11-17 01:51:06,19
jg,object,3967851.0,0,2,,,,,,阴性,3966819


**数据说明**
- sno：序列号，用于唯一标识一条核酸采样记录。
- user_id：人员 ID，对应于“人员信息表”中的 user_id，用于关联核酸采样记录与相应的人员。
- cysj：采样日期和时间，用于记录核酸采样的日期和时间。
- jcsj：检测日期和时间，用于记录核酸检测的日期和时间。
- jg：检测结果，用于记录核酸检测的结果。可选值为：阴性、阳性、未知。
- grid_point_id：场所 ID，对应于“场所信息表”中的 grid_point_id，用于关联核酸采样记录与相应的场所。


**以下分析结果均基于全量数据**

核酸采样记录总共有3,967,851条数据。

- 采样时间（cysj）与检测时间（jcsj）的关系：按照逻辑来说，检测时间应该晚于采样时间。可以通过对比这两个时间来判断数据是否异常。

    Tips:在实际数据分析中，可能会出现异常值，如检测时间早于采样时间的情况。这些异常值需要在分析时进行处理。

- 检测结果（jg）分布：示例数据中检测结果均为阴性。

    Tips:实际数据中可能包含阴性、阳性和未知三种结果。在进行数据分析时，需要考虑各种结果的分布情况。

根据上述分析，我们可以得出以下建议：

对比采样时间（cysj）与检测时间（jcsj），找出可能的异常值并进行处理。
分析检测结果（jg）的分布情况，了解阴性、阳性和未知结果的比例。
结合场所信息和人员信息，分析不同场所、人群的核酸检测情况，以了解场所类型和人群特征对核酸检测的影响。
分析采样和检测时间与其他表的时间信息，如扫码记录时间和自查上报时间，以了解时间因素在核酸检测过程中的作用。

五个数据表的结构和内容，分别为：场所信息表、个人自查上报信息表、扫码记录表、核酸采样记录表和体温测量记录表。

场所信息表：包含了场所的 ID、名称、类型、坐标和记录创建时间等信息。可以通过此表了解不同类型场所的分布情况。

个人自查上报信息表：包含了自查记录序列号、人员 ID、症状、核酸检测结果、居住情况、上报地点坐标和上报时间等信息。可以通过此表了解人员的自查症状和核酸检测结果等情况。

扫码记录表：包含了扫码记录序列号、场所 ID、人员 ID、体温和记录创建时间等信息。可以通过此表了解人员在不同场所的扫码记录和体温情况。

核酸采样记录表：包含了采样记录序列号、人员 ID、采样时间、检测时间、检测结果和场所 ID 等信息。可以通过此表了解人员的核酸采样和检测情况。

体温测量记录表：包含了测量记录序列号、场所 ID、人员 ID、体温和记录创建时间等信息。可以通过此表了解人员的体温测量情况。


为了更深入地了解疫情状况，我们可以将这些表进行关联分析，比如分析不同类型场所中人员的症状、核酸检测结果和体温情况，或者结合时间因素分析疫情的发展趋势等。这些分析结果可以为疫情防控提供有价值的参考。

# 数据清洗

## 对 df_people 数据进行清洗

In [34]:
df_people = read_file('data/all/附件2.csv')
print(f"清洗前的数据量: {df_people.shape[0]}")

# 删除年龄异常值
print(f"100岁以上：{(df_people['age'] > 100).sum()}\n115岁以上：{(df_people['age'] > 115).sum()}")
df_people = df_people[(df_people['age'] >= 0) & (df_people['age'] <= 115)]

# 用众数填充性别缺失值
col = 'gender'
missing_values_before = df_people[col].isna().sum()
df_people[col].fillna(df_people[col].mode()[0], inplace=True)
missing_values_after = df_people[col].isna().sum()
print(f"处理{col}缺失值前后数量变化: {missing_values_before} -> {missing_values_after}")

# 处理birthdate缺失值
col = 'birthdate'
missing_values_before = df_people[col].isna().sum()
current_year = pd.to_datetime('today').year
df_people.loc[df_people[col].isna(), col] = df_people.loc[df_people[col].isna(), 'age'].apply(lambda x: pd.Timestamp(year=int(current_year-x), month=1, day=1))
missing_values_after = df_people[col].isna().sum()
print(f"处理{col}缺失值前后数量变化: {missing_values_before} -> {missing_values_after}")


# 转换birthdate列的数据类型为日期类型
df_people['birthdate'] = pd.to_datetime(df_people['birthdate'])
# 转换create_time列的数据类型为日期类型
df_people['create_time'] = pd.to_datetime(df_people['create_time'])

# 删除年龄与birthdate和create_time不匹配的行
mismatch_before = df_people.shape[0]
df_people['year_difference'] = (df_people['create_time'] - df_people['birthdate']).dt.days // 365
df_people = df_people[abs(df_people['year_difference'] - df_people['age']) <= 2]
mismatch_after = df_people.shape[0]
print(f"处理年龄与birthdate和create_time不匹配的行前后数量变化: {mismatch_before} -> {mismatch_after}")
df_people.drop(columns=['year_difference'], inplace=True)

# 计算重复值并在处理过程中输出
for col in ['openid', 'user_id', 'create_time']:
    duplicate_values = df_people[df_people[col].duplicated(keep=False)]
    print(f"重复的{col}数量: {duplicate_values.shape[0]}")
    df_people = df_people[~df_people[col].duplicated(keep='first')]

# 保存清洗后的数据
df_people.to_csv('data/cleaned/人员信息表.csv', index=False)

# 输出清洗后的数据量
print(f"清洗后的数据量: {df_people.shape[0]}")

Reading file: data/all/附件2.csv
清洗前的数据量: 99996
100岁以上：13
115岁以上：13
处理gender缺失值前后数量变化: 31 -> 0
处理birthdate缺失值前后数量变化: 3 -> 0
处理年龄与birthdate和create_time不匹配的行前后数量变化: 99966 -> 99965
重复的openid数量: 1968
重复的user_id数量: 0
重复的create_time数量: 154
清洗后的数据量: 97921


## 对 df_place 数据进行清洗（数据干净）

In [18]:
df_place = read_file('data/all/附件3.csv')

# 将create_time转换为日期类型
df_place['create_time'] = pd.to_datetime(df_place['create_time'])

# 删除重复行
df_place = df_place.drop_duplicates()

# 检查是否有缺失值
print("缺失值情况：")
print(df_place.isnull().sum())

# 如果有缺失值，可以考虑删除缺失值所在的行或使用合适的方法填充缺失值
# 假设这里没有缺失值，无需处理

# 保存清洗后的数据到新的CSV文件
df_place.to_csv('data/cleaned/场所信息表.csv', index=False)

Reading file: data/all/附件3.csv
缺失值情况：
grid_point_id    0
name             0
point_type       0
x_coordinate     0
y_coordinate     0
create_time      0
dtype: int64


## 对 df_self_check 数据进行清洗（数据干净）

In [19]:
df_self_check = read_file('data/all/附件4.csv')
# 将dump_time转换为日期类型
df_self_check['dump_time'] = pd.to_datetime(df_self_check['dump_time'])

# 删除重复行
df_self_check = df_self_check.drop_duplicates()

# 检查缺失值
print("缺失值情况：")
print(df_self_check.isnull().sum())

# 处理异常值（如果有）
# 例如：处理年龄为负数的异常值
# df_self_check = df_self_check[df_self_check['age'] >= 0]

# 保存清洗后的数据到新的CSV文件
df_self_check.to_csv('data/cleaned/个人自查上报信息表.csv', index=False)

Reading file: data/all/附件4.csv
缺失值情况：
sno                     0
user_id                 0
x_coordinate            0
y_coordinate            0
symptom                 0
nucleic_acid_result     0
resident_flag           0
dump_time               0
dtype: int64


## 对 df_scan 数据进行清洗

In [11]:
import glob
import os
import re
from datetime import datetime

file_list = glob.glob('data/all/附件5/*.csv')

for file_path in file_list:
    df_scan = read_file(file_path)
    df_scan['create_time'] = pd.to_datetime(df_scan['create_time'])

    missing_values_before = df_scan.isnull().sum().sum()
    df_scan = df_scan.dropna()
    missing_values_after = df_scan.isnull().sum().sum()
    print(f"处理缺失值前后数量变化: {missing_values_before} -> {missing_values_after}")

    # 处理重复值
    duplicates_before = df_scan.duplicated(subset=['user_id', 'grid_point_id', 'create_time']).sum()
    df_scan = df_scan.drop_duplicates(subset=['user_id', 'grid_point_id', 'create_time'], keep='first')
    duplicates_after = df_scan.duplicated(subset=['user_id', 'grid_point_id', 'create_time']).sum()
    print(f"处理重复值前后数量变化: {duplicates_before} -> {duplicates_after}")

    # 处理异常值，体温范围
    temp_outliers_before = ((df_scan['temperature'] < 35.0) | (df_scan['temperature'] > 42.0)).sum()
    df_scan = df_scan[(df_scan['temperature'] >= 35.0) & (df_scan['temperature'] <= 42.0)]
    temp_outliers_after = ((df_scan['temperature'] < 35.0) | (df_scan['temperature'] > 42.0)).sum()
    print(f"处理异常体温值前后数量变化: {temp_outliers_before} -> {temp_outliers_after}")
    # 删除与文件名中日期不匹配的行
    date_pattern = r'\d{8}'
    date_str = re.search(date_pattern, file_path).group(0)
    file_date = datetime.strptime(date_str, '%Y%m%d').date()
    date_mismatch_before = (df_scan['create_time'].dt.date != file_date).sum()
    df_scan = df_scan[df_scan['create_time'].dt.date == file_date]
    date_mismatch_after = (df_scan['create_time'].dt.date != file_date).sum()
    print(f"处理日期不匹配的行前后数量变化: {date_mismatch_before} -> {date_mismatch_after}")


    # 保存清洗后的数据到新的CSV文件
    cleaned_file_path = os.path.join('data', 'cleaned', '场所扫码信息表',os.path.basename(file_path))
    df_scan.to_csv(cleaned_file_path, index=False)
    print(f"已保存清洗后的数据到文件: {cleaned_file_path}\n")

Reading file: data/all/附件5\20221001.csv
处理缺失值前后数量变化: 0 -> 0
处理重复值前后数量变化: 0 -> 0
处理异常体温值前后数量变化: 0 -> 0
处理日期不匹配的行前后数量变化: 194 -> 0
已保存清洗后的数据到文件: data\cleaned\场所扫码信息表\20221001.csv

Reading file: data/all/附件5\20221002.csv
处理缺失值前后数量变化: 0 -> 0
处理重复值前后数量变化: 0 -> 0
处理异常体温值前后数量变化: 0 -> 0
处理日期不匹配的行前后数量变化: 37691 -> 0
已保存清洗后的数据到文件: data\cleaned\场所扫码信息表\20221002.csv

Reading file: data/all/附件5\20221003.csv
处理缺失值前后数量变化: 0 -> 0
处理重复值前后数量变化: 0 -> 0
处理异常体温值前后数量变化: 0 -> 0
处理日期不匹配的行前后数量变化: 29640 -> 0
已保存清洗后的数据到文件: data\cleaned\场所扫码信息表\20221003.csv

Reading file: data/all/附件5\20221004.csv
处理缺失值前后数量变化: 0 -> 0
处理重复值前后数量变化: 0 -> 0
处理异常体温值前后数量变化: 0 -> 0
处理日期不匹配的行前后数量变化: 20667 -> 0
已保存清洗后的数据到文件: data\cleaned\场所扫码信息表\20221004.csv

Reading file: data/all/附件5\20221005.csv
处理缺失值前后数量变化: 0 -> 0
处理重复值前后数量变化: 0 -> 0
处理异常体温值前后数量变化: 0 -> 0
处理日期不匹配的行前后数量变化: 22260 -> 0
已保存清洗后的数据到文件: data\cleaned\场所扫码信息表\20221005.csv

Reading file: data/all/附件5\20221006.csv
处理缺失值前后数量变化: 0 -> 0
处理重复值前后数量变化: 0 -> 0
处理异常体温值前后数量变化: 0 -> 0
处理日期不

## 对 df_na_check 数据进行清洗

In [21]:
df_na_check = read_file('data/all/附件6.csv')

# 将cysj和jcsj转换为日期类型
df_na_check['cysj'] = pd.to_datetime(df_na_check['cysj'])
df_na_check['jcsj'] = pd.to_datetime(df_na_check['jcsj'])

# 删除重复行
df_na_check = df_na_check.drop_duplicates()

# 检查缺失值
print("缺失值情况：")
print(df_na_check.isnull().sum())

# 处理异常值（如果有）
# 例如：处理jcsj早于cysj的异常值
df_na_check = df_na_check[df_na_check['jcsj'] >= df_na_check['cysj']]

# 保存清洗后的数据到新的CSV文件
df_na_check.to_csv('data/cleaned/核酸检测信息表.csv', index=False)

Reading file: data/all/附件6.csv
缺失值情况：
sno              0
user_id          0
cysj             0
jcsj             0
jg               0
grid_point_id    0
dtype: int64


## 对 df_vaccine 数据进行清洗

In [23]:
df_vaccine = read_file('data/all/附件7.csv')

# 处理缺失值
missing_values_before = df_vaccine.isnull().sum().sum()
df_vaccine = df_vaccine.dropna(subset=['inject_sn', 'user_id', 'age', 'gender', 'birthdate', 'inject_date', 'inject_times', 'vaccine_type'])
missing_values_after = df_vaccine.isnull().sum().sum()
print(f"处理缺失值前后数量变化: {missing_values_before} -> {missing_values_after}")

# 检查并处理重复值
duplicates_before = df_vaccine.duplicated(subset=['inject_sn']).sum()
df_vaccine = df_vaccine.drop_duplicates(subset=['inject_sn'], keep='first')
duplicates_after = df_vaccine.duplicated(subset=['inject_sn']).sum()
print(f"处理重复值前后数量变化: {duplicates_before} -> {duplicates_after}")

# 检查并处理异常值，如：年龄、疫苗类型、接种次数等
age_outliers_before = ((df_vaccine['age'] < 0) | (df_vaccine['age'] > 120)).sum()
df_vaccine = df_vaccine[(df_vaccine['age'] >= 0) & (df_vaccine['age'] <= 120)]
age_outliers_after = ((df_vaccine['age'] < 0) | (df_vaccine['age'] > 120)).sum()
print(f"处理异常年龄值前后数量变化: {age_outliers_before} -> {age_outliers_after}")

# 将日期列转换为 datetime 类型
df_vaccine['birthdate'] = pd.to_datetime(df_vaccine['birthdate'])
df_vaccine['inject_date'] = pd.to_datetime(df_vaccine['inject_date'])

# 保存清洗后的数据到新的CSV文件
df_vaccine.to_csv('data/cleaned/疫苗接种信息表.csv', index=False)

Reading file: data/all/附件7.csv
处理缺失值前后数量变化: 1107 -> 0
处理重复值前后数量变化: 160 -> 0
处理异常年龄值前后数量变化: 33 -> 0


# TODO

数据整合：将关联的表进行合并，例如将场所扫码信息表与场所信息表、人员信息表和个人自查上报信息表进行合并，方便后续分析。

特征工程：根据业务需求，从原始数据中提取、构建有意义的特征，例如从时间信息中提取星期、小时等时间特征，计算每个人在不同场所的扫码次数等。

数据分析：对整合后的数据进行描述性统计、可视化等探索性数据分析，以了解数据的基本情况、发现数据中的规律和异常。

建模预测：基于数据分析的结果，选择合适的模型进行训练，以解决实际问题，例如预测某个人员是否感染、预测核酸检测结果等。

模型评估与优化：评估模型的性能，如准确率、召回率等，并通过调整模型参数、特征选择等方法进行优化。

结果呈现：将分析结果以可视化、报告等形式呈现。

In [ ]:
#数据整合并提取特征
df_na_check = read_file('data/all/附件6.csv')
file_list = glob.glob('data/all/附件5/*.csv')
df_scan = pd.concat([read_file(file) for file in file_list])
df_scan['create_time'] = pd.to_datetime(df_scan['create_time'])
df_people = read_file('data/all/附件2.csv')
df_place = read_file('data/all/附件3.csv')
df_self_check = read_file('data/all/附件4.csv')
df_self_check['dump_time'] = pd.to_datetime(df_self_check['dump_time'])

# 数据整合
df_scan_place = pd.merge(df_scan, df_place, on='grid_point_id', suffixes=('_scan', '_place'))
df_self_check_people = pd.merge(df_self_check, df_people, on='user_id', suffixes=('_self_check', '_people'))

# 特征工程
# 提取扫码时间的小时特征
df_scan_place['hour'] = df_scan_place['create_time'].dt.hour

# 计算每个人在不同场所的扫码次数
scan_count_by_user_place = df_scan_place.groupby(['user_id', 'grid_point_id']).size().reset_index(name='scan_count')

# 合并特征到主数据集
df_scan_place = pd.merge(df_scan_place, scan_count_by_user_place)

**数据分析时可以做以下可视化**

**单表可视化**

1. 人员信息表：可以进行人口统计学分析，如性别、年龄、民族等分布情况，还可以通过人员 ID 与其他表格进行关联分析。

2. 场所信息表：可以进行地理信息分析，如场所分布情况、场所类型分布情况、场所密度等分析。

3. 个人自查上报信息表：可以进行疫情监测分析，如症状分布情况、症状与核酸检测结果的关联分析、上报人员的位置分布情况等分析。

4. 场所码扫码信息表：可以进行疫情监测分析，如扫码记录分布情况、扫码记录与核酸检测结果的关联分析等。

5. 核酸采样检测信息表：可以进行疫情监测分析，如阳性人员的分布情况、核酸检测阳性率分析、阳性人员的接触场所与密切接触者分析等。


**关联分析**

1. 个人自查上报信息表和核酸采样检测信息表：可以分析个人上报的症状与核酸检测结果之间的关系，以及症状与检测结果对不同年龄、性别、民族等人群的影响。

2. 场所信息表和场所码扫码信息表：可以分析不同场所的扫码情况，了解人们在哪些场所更容易扫码；也可以分析场所内体温异常者的情况，了解哪些场所的防疫工作存在漏洞。

3. 个人自查上报信息表和场所码扫码信息表：可以根据个人自查上报的症状，分析不同场所的症状发生情况，了解哪些场所的防疫措施需要进一步加强。

4. 核酸采样检测信息表和个人自查上报信息表、场所码扫码信息表：可以分析阳性人员的出行情况，追踪密接者，及时采取隔离措施。

Task3

In [ ]:
#第一步：确定每个场所的温度分布的平均值和标准偏差
df_scan['temperature_mean'] = df_scan.groupby('grid_point_id')['temperature'].transform('mean') #平均值
df_scan['temperature_std'] = df_scan.groupby('grid_point_id')['temperature'].transform('std') #方差

In [ ]:
#根据场所码扫码信息表和核酸采样信息表，确定感染者的平均体温
#df_na_check是另一个数据框，其中包含有关用户的信息，包括他们是否对某种疾病进行了阳性或阴性测试。这里，df_na_check['jg'] == '阳性用于过滤出仅对该疾病测试呈阳性的用户。然后，[['user_id']]被用来仅选择从过滤后的df_na_check数据框中的'user_id'列。
#使用pd.merge将df_scan和过滤后的df_na_check数据框基于'user_id'列进行连接，只有在结果数据框df_positive中有匹配'user_id'值的行被包含进来。
#最后，使用df_positive.groupby('grid_point_id')['temperature'].transform('mean')计算具有相同'grid_point_id'值的行组的平均温度值。结果是一个名为'temperature_mean_positive'的新列，其中包含了df_positive数据框中每个行组的平均温度值。
#因此，这段代码创建了一个名为df_positive的新数据框，其中包含了仅对某种疾病测试呈阳性的用户的df_scan行，并计算了每个行组的平均温度值，并将这些信息作为新列添加到了df_positive数据框中。
df_positive = pd.merge(df_scan, df_na_check[df_na_check['jg'] == '阳性'][['user_id']], on='user_id', how='inner')
df_positive['temperature_mean_positive'] = df_positive.groupby('grid_point_id')['temperature'].transform('mean')
print(df_positive)

           sno  grid_point_id  user_id  temperature         create_time  \
0          283           2285    10063         37.2 2022-10-01 18:50:02   
1          284           2649    10063         36.9 2022-10-01 17:34:02   
2          285           2232    10063         37.2 2022-10-01 19:10:02   
3          291           1439    10063         36.7 2022-10-02 11:28:11   
4          292           2070    10063         36.9 2022-10-02 16:23:11   
...        ...            ...      ...          ...                 ...   
231068  317859           1305    79358         37.1 2022-11-29 17:26:49   
231069  317850            214    79358         35.7 2022-11-30 08:57:03   
231070  317851           2766    79358         36.2 2022-11-30 12:21:03   
231071  317852             49    79358         35.9 2022-11-30 15:32:03   
231072  317853           2170    79358         37.0 2022-11-30 19:15:03   

        temperature_mean  temperature_std  temperature_mean_positive  
0              36.453399    

In [ ]:
#这段代码是在前面的代码基础上进行计算，它将计算结果添加到了df_positive数据框中的新一列infection_prob中。
#具体来说，np.exp()是numpy库中的一个函数，它返回e的x次方。在这里，它被用于计算指数值。
#接下来，代码中的数学公式((df_positive['temperature_mean_positive'] - df_positive['temperature_mean']) ** 2) / (2 * df_positive['temperature_std'] ** 2)用于计算指数中的底数。它计算了两个温度均值之间的平方差，并除以两个温度标准差的平方之和，从而得到一个浮点数。
#最后，这个浮点数作为指数的底数被传递给np.exp()函数中，从而得到最终的infection_prob值。这个值表示在特定条件下，用户被感染的概率。
###这个计算公式的目的是通过测量阳性用户体温与所有用户体温的差异，来推测用户是否被感染。这个差异越大，表示阳性用户体温相对于所有用户的体温有着更大的偏离，从而意味着阳性用户更可能被感染。
###具体地，这个公式计算的是两个均值之间的平方差，然后除以两个标准差的平方之和。这个值表示阳性用户体温相对于所有用户的体温的差异程度，差异越大，值越大，从而表示用户被感染的概率也越大。
df_positive['infection_prob'] = np.exp(-((df_positive['temperature_mean_positive'] - df_positive['temperature_mean']) ** 2) / (2 * df_positive['temperature_std'] ** 2))
print(df_positive)

           sno  grid_point_id  user_id  temperature         create_time  \
0          283           2285    10063         37.2 2022-10-01 18:50:02   
1          284           2649    10063         36.9 2022-10-01 17:34:02   
2          285           2232    10063         37.2 2022-10-01 19:10:02   
3          291           1439    10063         36.7 2022-10-02 11:28:11   
4          292           2070    10063         36.9 2022-10-02 16:23:11   
...        ...            ...      ...          ...                 ...   
231068  317859           1305    79358         37.1 2022-11-29 17:26:49   
231069  317850            214    79358         35.7 2022-11-30 08:57:03   
231070  317851           2766    79358         36.2 2022-11-30 12:21:03   
231071  317852             49    79358         35.9 2022-11-30 15:32:03   
231072  317853           2170    79358         37.0 2022-11-30 19:15:03   

        temperature_mean  temperature_std  temperature_mean_positive  \
0              36.453399   

In [ ]:
#将result1数据框按照['阳性人员ID', '密接场所ID']两列进行分组，统计了每个阳性人员在每个密接场所中的密接者数量，并将统计结果保存到了grouped数据框中。
grouped = result1.groupby(['阳性人员ID', '密接场所ID'])['密接者ID'].count().reset_index()
#接下来，将grouped数据框与result1数据框进行左连接，并将结果保存到result1_count数据框中。这个连接操作的目的是将每个阳性人员在每个密接场所中的密接者数量添加到原数据集result1中。
result1 = pd.merge(result1, grouped, on=['阳性人员ID', '密接场所ID'], how='left')
result1_count = result1.rename(columns={'密接者ID_y': '密接者数量'})
result1_count['平均接触人数'] = result1_count.groupby(['阳性人员ID', '密接场所ID'])['密接者数量'].transform('mean')
#最后，将result1_count数据框与df_positive数据框进行连接，并将结果保存到df_positive数据框中。这个连接操作的目的是将每个阳性人员在每个密接场所中的密接者数量和平均接触人数添加到df_positive数据框中，以便后续分析。
df_positive = pd.merge(result1_count, df_positive, left_on='阳性人员ID', right_on='user_id')
print(df_positive)

          序号  密接者ID_x                 密接日期  密接场所ID  阳性人员ID  密接者数量  平均接触人数  \
0          1    64306  2022-11-19 14:47:33     520   78049     22    22.0   
1          1    64306  2022-11-19 14:47:33     520   78049     22    22.0   
2          1    64306  2022-11-19 14:47:33     520   78049     22    22.0   
3          1    64306  2022-11-19 14:47:33     520   78049     22    22.0   
4          1    64306  2022-11-19 14:47:33     520   78049     22    22.0   
...      ...      ...                  ...     ...     ...    ...     ...   
343747  1291    44049  2022-11-04 12:23:33    2040    5415      3     3.0   
343748  1291    44049  2022-11-04 12:23:33    2040    5415      3     3.0   
343749  1291    44049  2022-11-04 12:23:33    2040    5415      3     3.0   
343750  1291    44049  2022-11-04 12:23:33    2040    5415      3     3.0   
343751  1291    44049  2022-11-04 12:23:33    2040    5415      3     3.0   

           sno  grid_point_id  user_id  temperature         create_time  \


In [ ]:
#这段代码的作用是为df_positive数据框添加一列名为label的新列，该列的值是df_positive数据框中infection_prob和平均接触人数两列的乘积。infection_prob表示该用户被感染的概率，平均接触人数表示该用户平均接触的人数。因此，label列的值表示该用户在疫情传播中的风险值。如果label的值越大，则表示该用户在疫情传播中的风险越高。
df_positive['label'] = df_positive['infection_prob'] * df_positive['平均接触人数']

In [ ]:
# 疫苗接种信息表
df_vaccine_info = read_file('cleaned/疫苗接种信息表.csv')
df = pd.merge(df_vaccine_info, df_positive, on='user_id')
# 去除没有label的数据
df = df.dropna()

Reading file: cleaned/疫苗接种信息表.csv


In [ ]:
col = ['age', 'gender','inject_times', 'vaccine_type','label']
df = df[col]
#这段代码导入了两个Python库：sklearn.preprocessing和sklearn.ensemble。其中，LabelEncoder是一个用于对标签进行编码的类，RandomForestRegressor是一个随机森林回归器，用于对数据进行回归分析。
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

# 创建 LabelEncoder 对象
le = LabelEncoder()

# 对 inject_times 和 vaccine_type 进行数值编码
df['inject_times'] = le.fit_transform(df['inject_times'])
df['vaccine_type'] = le.fit_transform(df['vaccine_type'])

# 创建随机森林回归模型
rf = RandomForestRegressor(n_estimators=100, random_state=2023)
# 拟合数据
X= df.drop('label',axis=1)
y = df['label']
rf.fit(X, y)

#这段代码计算了一个名为importances的变量，该变量存储了随机森林回归器rf中每个特征的重要性分数。随机森林回归器可以用于特征选择，feature_importances_属性可以返回每个特征的重要性分数，这些分数可以用于确定哪些特征对预测结果影响最大。importances变量是一个包含了每个特征重要性分数的数组。
importances = rf.feature_importances_

# 将特征重要性排序
indices = np.argsort(importances)[::-1]

# 将特征名称按照重要性排序
names = [f'Feature {i}' for i in range(X.shape[1])]
sorted_names = [names[i] for i in indices]

# 绘制特征重要性柱状图
plt.figure()
plt.title("Feature Importance")
plt.bar(range(X.shape[1]), importances[indices])
plt.xticks(range(X.shape[1]), sorted_names, rotation=90)
plt.show()